### Task
* Given
    * Operating temp & pressure (K, atm)
    * Reaction Kinetics Information
    * Reaction Thermochemistry Information (MJ/kmol)
    * Reaction Stoichiometry
    * Composition at inlet (kmol/m3)
<br>
* To find
    * Composition at outlet (kmol/m3)
    * Cooling/Heating duty (MJ) if reactor is isothermal

In [ ]:
import pandas as pd
import math
from typing import Callable
import pfr_utils as pu

#### 1.1. Steady-State Simulation using Euler's finite difference method
* 2A + B = C + 3D

In [ ]:
moles_A = {0.00:0.8} # Keys will be z-co-ordinate
moles_B = {0.00:0.5}
moles_C = {0.00:0}
moles_D = {0.00:0}

Temp_kelvin = {0.00:900}

if cooling is True:
    rate_const_entry = rate_const(Temp_kelvin[list(Temp_kelvin.keys())[0]])

# For 10 nodes
for i in range(10):
    z_coord = float(f"{(i*length_step):.2f}")
    z_next = float(f"{(z_coord + length_step):.2f}")
    
    # Calculate rate of reaction in this element w.r.t. A using temperature at 
    # entry of elemental volume
    if cooling is False:
        rate_A = rate_const(Temp_kelvin[z_coord]) * (moles_A[z_coord] ** abs(stoic_cf[0]))
    else:
        rate_A = rate_const_entry * (moles_A[z_coord] ** abs(stoic_cf[0]))
    
    # Calculate concentration to check for limiting reactant
    moles_reacted_A = (rate_A * (length_step ** 2)/(axial_vel))
    next_moles_A = moles_A[z_coord] - moles_reacted_A

    next_moles_B = moles_B[z_coord] - \
                    (stoic_cf[1]/stoic_cf[0]) * moles_reacted_A
    
    next_moles_C = moles_C[z_coord] - \
                    (stoic_cf[2]/stoic_cf[0]) * moles_reacted_A
    
    next_moles_D = moles_D[z_coord] - \
                    (stoic_cf[3]/stoic_cf[0]) * moles_reacted_A

    # Check for limiting reactant. Only proceed if no concentration is equal to 0
    if next_moles_A > 0 and next_moles_B > 0:
        # Calculate moles for next node
        moles_A[z_next] = next_moles_A
        moles_B[z_next] = next_moles_B
        moles_C[z_next] = next_moles_C
        moles_D[z_next] = next_moles_D

        # Calculate temperature for next node & cooling duty as per settings of reactor cooling
        if cooling is False:
            
            # Calculate density & specific heat capacity
            moles_state = [
                moles_A[z_coord],moles_B[z_coord],
                moles_C[z_coord],moles_D[z_coord]
                ]
            element_mass = sum([a*b 
                                for a,b in zip(moles_state,molecular_wt)
                                ])
            element_sp_heat_cap = sum([
                a*b*c*1e-3 
                for a,b,c in zip(moles_state,sp_heat_cap,molecular_wt)
                ])/element_vol
            
            # Calculate temperature for next node
            Temp_kelvin[z_next] = Temp_kelvin[z_coord] + (moles_reacted_A*heat_of_rxn/\
                                                      (element_mass*element_sp_heat_cap))
            
        else:
            cooling_duty_kJ += moles_reacted_A*heat_of_rxn
            Temp_kelvin[z_next] = Temp_kelvin[z_coord]
    
    # Else if some concentration is zero
    else:
        # Determine moles and temperature for next node, as being the same as current node
        moles_A[z_next] = moles_A[z_coord]
        moles_B[z_next] = moles_B[z_coord]
        moles_C[z_next] = moles_C[z_coord]
        moles_D[z_next] = moles_D[z_coord]
        Temp_kelvin[z_next] = Temp_kelvin[z_coord]
        

In [ ]:
df_moles = pd.DataFrame(data=[moles_A,moles_B,moles_C,moles_D],
                       index=["moles_A","moles_B","moles_C","moles_D"]).transpose()
df_moles

In [ ]:
from matplotlib_funcs import *

plot_matplotlib_style(df_moles.index,df_moles["moles_A"],"Length","Moles of A (mol)","Moles of A (mol)")
plot_matplotlib_style(df_moles.index,df_moles["moles_B"],"Length","Moles of B (mol)","Moles of B (mol)")
plot_matplotlib_style(df_moles.index,df_moles["moles_C"],"Length","Moles of C (mol)","Moles of C (mol)")
plot_matplotlib_style(df_moles.index,df_moles["moles_D"],"Length","Moles of D (mol)","Moles of D (mol)")
plot_matplotlib_style(df_moles.index,Temp_kelvin.values(),"Length","Temperature (K)","Temperature (K)")

In [ ]:
cooling_duty_kJ

#### 1.2. Transient Simulation using Euler's Finite Difference Method
* Transient Mass Balance
* Transient Energy Balance
* [Tutorial: Numerical Solution of PDEs by FDM](https://math.libretexts.org/Bookshelves/Differential_Equations/Introduction_to_Partial_Differential_Equations_(Herman)/10%3A_Numerical_Solutions_of_PDEs/10.02%3A_The_Heat_Equation)

In [51]:
from pprint import pprint
import pfr_calc as pc
import pfr_utils as pu
import math
import json

# run function
def run()->dict[str, dict[float, float]]:
    # Configure the individual config elements separately, and then assemble
    # in ProcessSimulationConfig object
    Chem_A = pu.Chemical("A",20,4)
    Chem_B = pu.Chemical("B",20,5)
    Chem_C = pu.Chemical("C",30,2)
    Chem_D = pu.Chemical("D",10,10)
    
    my_reaction = pu.Reaction({Chem_A:-2,Chem_B:-1,Chem_C:1,Chem_D:3},50000,2e-3,5)
    my_reactor = pu.Reactor(False,0.4,2)
    my_solver = pu.ReactorFDMSolver(1e-2,0.1)

    my_config =pu.ProcessSimulationConfig(
        reaction=my_reaction,
        reactor=my_reactor,
        solver=my_solver
    )
    
    entry_mol = [1,0.8,0,0.1]
    axial_vel = 1e-4
    t_points = [float(f"{(i*math.pi/10):.2f}") for i in range(10)]  
    return pc.calc_steady_profile(my_config,entry_mol,axial_vel)

output_1 = run()

In [55]:
# TODO: Recursively convert all values to strings to save into json

def stringify(dict1:dict):
    for key, val in dict1.items():
        if type(val) is dict:
            stringify(val)
        elif type(val) is list:
            # list of dict
            if type(val[0]) is dict:
                [stringify(value_dict)
                 for value_dict in val]

            dict1.update({key:str(val)})
    return dict1

new_dict = stringify(output_1)
pprint(new_dict)

{Chemical(name='D', mol_wt=10, sp_heat_cap=10): "<class 'list'>",
 Chemical(name='A', mol_wt=20, sp_heat_cap=4): "<class 'list'>",
 Chemical(name='B', mol_wt=20, sp_heat_cap=5): "<class 'list'>",
 Chemical(name='C', mol_wt=30, sp_heat_cap=2): "<class 'list'>",
 'Temperature (K)': {0.0: 900,
                     0.1: 900.0,
                     0.2: 900.0,
                     0.3: 900.0,
                     0.4: 900.0,
                     0.5: 900.0,
                     0.6: 900.0,
                     0.7: 900.0,
                     0.8: 900.0,
                     0.9: 900.0,
                     1.0: 900.0}}


In [52]:
# Run the run function with output formatting & saving in json format
def run_save(**kwargs):
    stringify(output_1)
    
    # Choose between json & pprint for testing. TODO: Delete after json is created.
    if kwargs["flag"] == 'json':
        with open("steady1.json",'w') as f:
            json.dump(json.loads(output_1),f,indent=4)
    elif kwargs["flag"] == 'pprint':
        pprint(output_1)

run_save(flag="json")

TypeError: the JSON object must be str, bytes or bytearray, not dict